In [1]:
from langchain_groq import ChatGroq

In [2]:
llm = ChatGroq(
    temperature=0, 
    groq_api_key='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', 
    model_name="llama-3.1-70b-versatile"
)
response = llm.invoke("The first person to land on moon was ...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped onto the lunar surface on July 20, 1969, as part of the Apollo 11 mission.


In [10]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-37695?from=job%20search%20funnel")
page_data = loader.load().pop().page_content
print(page_data)

Apply for Manager Software Engineering, ITC

Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsTechnologyManager Software Engineering, ITCKarnataka, IndiaBecome a Part of the NIKE, Inc. Te

In [15]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [16]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Manager Software Engineering, ITC',
 'experience': '12+ years of experience in software development, 5 years management experience',
 'skills': ['Academic background in computer science (BS or MS) or equivalent work experience',
  'Leadership in a highly collaborative, multi-discipline development team environment',
  'Exposure to backend development skills in Java/Spring',
  'Experience of building microservices using cloud services to build an integrated application (AWS preferred)',
  'Exposure to tuning relational database (MySQL) and non-relation DB (MongoDB, Dynamo DB)',
  'Experience with building data-intensive web applications using modern UI frameworks like React or Polymer',
  'Experience working with Product Managers, other development teams, and technology platform teams to accelerate software development and increase quality',
  'Experience in goal settings, mentoring hiring and ramping up the teams',
  'Comfortable working within an exciting fast-paced environm

In [13]:
type(json_res)

dict

In [17]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [18]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [33]:
links = collection.query(query_texts=job['skills'], n_results=1).get('metadatas', [])
links

[[{'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'}],
 [{'links': 'https://example.com/kotlin-backend-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'}],
 [{'links': 'https://example.com/react-portfolio'}],
 [{'links': 'https://example.com/react-portfolio'}],
 [{'links': 'https://example.com/xamarin-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/android-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/xamarin-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'}]]

In [26]:
job

{'role': 'Manager Software Engineering, ITC',
 'experience': '12+ years of experience in software development, 5 years management experience',
 'skills': ['Academic background in computer science (BS or MS) or equivalent work experience',
  'Leadership in a highly collaborative, multi-discipline development team environment',
  'Exposure to backend development skills in Java/Spring',
  'Experience of building microservices using cloud services to build an integrated application (AWS preferred)',
  'Exposure to tuning relational database (MySQL) and non-relation DB (MongoDB, Dynamo DB)',
  'Experience with building data-intensive web applications using modern UI frameworks like React or Polymer',
  'Experience working with Product Managers, other development teams, and technology platform teams to accelerate software development and increase quality',
  'Experience in goal settings, mentoring hiring and ramping up the teams',
  'Comfortable working within an exciting fast-paced environm

In [27]:
job = json_res
job['skills']

['Academic background in computer science (BS or MS) or equivalent work experience',
 'Leadership in a highly collaborative, multi-discipline development team environment',
 'Exposure to backend development skills in Java/Spring',
 'Experience of building microservices using cloud services to build an integrated application (AWS preferred)',
 'Exposure to tuning relational database (MySQL) and non-relation DB (MongoDB, Dynamo DB)',
 'Experience with building data-intensive web applications using modern UI frameworks like React or Polymer',
 'Experience working with Product Managers, other development teams, and technology platform teams to accelerate software development and increase quality',
 'Experience in goal settings, mentoring hiring and ramping up the teams',
 'Comfortable working within an exciting fast-paced environment by reducing ambiguity and building trust with the team',
 'Willingness to explore new technologies and creative solutions',
 'Strong at estimating work, plann

In [28]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Rasendra, a business development executive at RASENDRA_COMPANY. RASENDRA_COMPANY is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase RASENDRA_COMPANY's portfolio: {link_list}
        Remember you are Rasendra, BDE at RASENDRA_COMPANY. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Software Engineering Solutions for Nike's Digital Asset Management System

Dear Hiring Manager,

I came across the job description for the Manager Software Engineering, ITC role at Nike, and I'm excited to introduce AtliQ, an AI & Software Consulting company that can help you achieve your goals. With our expertise in software development, we can provide engineering leadership and best practices to continuously build new platforms and deliver working software.

Our team has extensive experience in building microservices using cloud services, specifically AWS, and has expertise in backend development skills in Java/Spring. We're also well-versed in tuning relational databases (MySQL) and non-relational DB (MongoDB, Dynamo DB). Our experience in building data-intensive web applications using modern UI frameworks like React or Polymer aligns perfectly with your requirements.

At AtliQ, we've worked on numerous projects that demonstrate our capabilities in software engineeri